# INSTALL
- WARNING ::: ONLY DO ONCE 
    - update devconfig in ~/.config/pyontutils/devonfig.yaml
        - scigraph_api: http://scigraph.scicrunch.io:9000/scigraph
    - Install both pyontutils and ilxutils with pyontutils
        - cd ~/git/pyontutils
        - pip3 install --user --editable .
        - cd ~/git/pyontutils/ilxutils/
        - pip3 install --user --editable .
    - Clone ontquery and install
        - cd ~/git
        - git clone https://github.com/tgbugs/ontquery.git
        - cd ~/git/ontquery
        - pip3 install --user --editable .

# Maintainance
- update repos
    - cd ~/git/pyontutils
    - git pull 
    - cd ~/git/ontquery
    - git pull

# Google Sheets Import
### Need pyontutils secrets.yaml setup first!

In [1]:
from pyontutils.sheets import Sheet
import pandas as pd
KEY_NAME = 'sparc-terms'
SHEET_NAME = 'Minimal information model(MIS)'

class Brainstem(Sheet):
    name = KEY_NAME # key name you gave the google sheet id value in secrets.yaml
    sheet_name = SHEET_NAME # the actual sheet name on the google sheet
    fetch_grid = True # meta data in self.grid that has detials like bolding

brainstem = Brainstem()
df = pd.DataFrame(brainstem.raw_values)
df.columns = df.iloc[0]
df.drop(df.index[0], inplace=True)
df.head()

AttributeError: 'NoneType' object has no attribute 'exists'

In [8]:
list(df.term)[:10]

['abdominal cavity',
 'abdominal wall',
 'adipose tissue',
 'adult organism',
 'alimentary part of gastrointestinal system',
 'arterial blood',
 'biceps femoris',
 'blood',
 'bolus of food',
 'brainstem']

# CSV or TSV  EXAMPLE

# QUERY DATABASES 

In [3]:
# Give "query" a usable parameter to query the databases 
from pyontutils.core import query # OntTerm
query(term='brain')

[OntTerm('HBA:3999', label='brain (hba)'),
 OntTerm('FMA:50801', label='Brain'),
 OntTerm('UBERON:0000955', label='brain'),
 OntTerm('UBERON:6110636', label='adult cerebral ganglion'),
 OntTerm('ILX:0101431', label='Brain'),
 OntTerm('ILX:0101433', label='Brain Infarction'),
 OntTerm('ILX:0506386', label='Brain Aneurysm'),
 OntTerm('ILX:0433050', label='Brain Chemistry'),
 OntTerm('ILX:0641746', label='alpha BRAIN'),
 OntTerm('ILX:0726394', label='brain meninx'),
 OntTerm('ILX:0729002', label='brain commissure'),
 OntTerm('ILX:0101434', label='Brain Ischemia'),
 OntTerm('ILX:0461406', label='Brain Death'),
 OntTerm('ILX:0733041', label='brain endothelium')]

In [4]:
# similar entities will show
# default limit is 10
query(term='DN1 neuron', limit=2) 

[OntTerm('ILX:0103358', label='DN1 neuron'),
 OntTerm('ILX:0109525', label='Pupal DN1 period neuron')]

In [5]:
# Faster and more accurate with curie/iri
query(curie='UBERON:0000955')

[OntTerm('UBERON:0000955', label='brain')]

In [6]:
entity = query(curie='UBERON:0000955')[0]
# Full result attribute
vars(entity)

{'prefix': 'UBERON',
 'suffix': '0000955',
 'orig_kwargs': {'iri': 'http://purl.obolibrary.org/obo/UBERON_0000955',
  'curie_or_iri': None,
  'label': None,
  'term': None,
  'search': None,
  'validated': None,
  'query': None},
 'kwargs': {'iri': 'http://purl.obolibrary.org/obo/UBERON_0000955',
  'curie_or_iri': None,
  'label': None,
  'term': None,
  'search': None,
  'validated': None,
  'query': None},
 'label': 'brain',
 'labels': ['brain'],
 'definition': 'The brain is the center of the nervous system in all vertebrate, and most invertebrate, animals. Some primitive animals such as jellyfish and starfish have a decentralized nervous system without a brain, while sponges lack any nervous system at all. In vertebrates, the brain is located in the head, protected by the skull and close to the primary sensory apparatus of vision, hearing, balance, taste, and smell[WP].',
 'synonyms': ['the brain',
  'synganglion',
  'suprasegmental structures',
  'suprasegmental levels of nervous s

# DEBUGGING HINT
- 1 "?" at the end of a function or class will return its params, docstring, and pathing. 
- 2 "??" returns the ENTIRE class/functions 

In [7]:
query?

Signature:      query(*args, **kwargs)
Call signature:
query(
    term=None,
    prefix=(),
    category=None,
    label=None,
    abbrev=None,
    search=None,
    suffix=None,
    curie=None,
    iri=None,
    predicates=(),
    exclude_prefix=(),
    depth=1,
    direction='OUTGOING',
    limit=10,
    include_deprecated=False,
)
Type:           OntQueryCli
String form:    <ontquery.query.OntQueryCli object at 0x7f13e5c53e10>
File:           ~/Dropbox/git/ontquery/ontquery/query.py
Docstring:      <no docstring>


# BONUS!

### Concurrently search! (Run multiple query functions at the same time)

In [8]:
from pyontutils.utils import Async, deferred
from pyontutils.core import OntTerm, ixr, query
from typing import List, Tuple

# query.setup()

def queries(kwargs_list:List[dict]) -> List[Tuple[str, dict]]:
    '''Asynchronously query databases to dramatically increase runtime un users end 
    
    Examples:
        >>> queries([{'term':'Brain'},])
        [({'term': 'Brain'},
          [OntTerm('HBA:3999', label='brain (hba)'),
           OntTerm('FMA:50801', label='Brain'),
           OntTerm('UBERON:0000955', label='brain'),
           OntTerm('UBERON:6110636', label='adult cerebral ganglion')])]
        >>> queries([{'curie':'UBERON:0000955'},])
        [({'curie': 'UBERON:0000955'}, [OntTerm('UBERON:0000955', label='brain')])]
        
    Definitions:
        kwargs == common name given to dictionary input for function
        tuple == a list that you cannot update. 
        lambda == short-hand for single line function creation (func = lambda <input>: <output>)  
        
    Args:
        kwargs_list (list): A list of dictionaries that are paramaters for the query function
    
    Returns:
        List[tuple]: A list of tuples all being of (<input>, <query_result>).    
    '''
    # create a query function wrapper to return tuple
    # kwargs -> (kwargs, query_result)
    # We do this in case 2+ queries return the same results & the output WILL NOT have the same input order
    gin = lambda kwargs: (kwargs, query(**kwargs))
    # run each query instance at the same time
    results = Async(use_nest_asyncio=True)(deferred(gin)(kwargs) for kwargs in kwargs_list)
    return results 

queries([{'curie':'UBERON:0000955'}, {'curie':'UBERON:6110636'}])

Futures compiled


[({'curie': 'UBERON:0000955'}, [OntTerm('UBERON:0000955', label='brain')]),
 ({'curie': 'UBERON:6110636'},
  [OntTerm('UBERON:6110636', label='adult cerebral ganglion')])]

In [ ]:
from pyontutils.utils import Async, deferred
from pyontutils.core import OntTerm, ixr, query
from typing import List, Tuple
def queries(url_list:List[dict]) -> List[Tuple[str, dict]]:
    def gin(url):
        return requests.get(url).text
    # run each query instance at the same time
    results = Async(limit=5)(deferred(gin)(url) for url in url_list)
    return results 
list_tuples(url, html)

# Modifing TEST InterLex

In [1]:
# TEST InterLex endpoints
from ilxutils.remotes import interlex_remote_test as ixrt

# GET ENTITY

In [2]:
ixrt.ilx_cli.get_entity('tmp_0738390')

{'id': '661544',
 'orig_uid': '34142',
 'uid': '34142',
 'orig_cid': '0',
 'cid': '0',
 'ilx': 'ilx_0738390',
 'label': 'Offical label',
 'type': 'term',
 'definition': 'official definition',
 'comment': 'helpful misc',
 'version': '3',
 'status': '0',
 'display_superclass': '1',
 'orig_time': '1564695195',
 'time': '1570826848',
 'synonyms': [{'id': '1776645',
   'tid': '661544',
   'literal': 'Encephalon',
   'type': '',
   'time': '1570826848',
   'version': '3'},
  {'id': '1776646',
   'tid': '661544',
   'literal': 'Cerebro',
   'type': '',
   'time': '1570826848',
   'version': '3'}],
 'superclasses': [],
 'existing_ids': [{'id': '3885545',
   'tid': '661544',
   'curie': 'ILX:0738390',
   'iri': 'http://uri.interlex.org/base/ilx_0738390',
   'curie_catalog_id': '3885424',
   'version': '3',
   'time': '1570826848',
   'preferred': '1'}],
 'relationships': [],
 'mappings': [],
 'annotations': [],
 'ontologies': []}

# ADD

In [2]:
ixrt.add_entity?

Signature:
ixrt.add_entity(
    type,
    subThingOf,
    label,
    definition: str = None,
    synonyms=(),
    comment: str = None,
    predicates: dict = None,
)
Docstring: <no docstring>
File:      ~/Dropbox/git/ontquery/ontquery/plugins/services.py
Type:      method


In [3]:
entity = dict(
    label = 'offical label', # Can only one unique label per person
    type = 'term', # OPTIONS: term, annotation, relationship, cde, fde, pde
    definition = 'official definition',
    comment = 'helpful misc',
    # Optional
    subThingOf = '', # WARNING ::: must have at last '', can be blank but please fill this in if you can. 
    synonyms = ['Encephalon', 'Cerebro'],
    predicates = {} # annotations and/or relationships to add
    # TODO: existing_ids will be an option later
)
result = ixrt.add_entity(**entity)
print(result)

QueryResult({'iri': 'http://uri.interlex.org/base/ilx_0738390', 'curie': 'ILX:0738390', 'label': 'official test label', 'labels': (), 'definition': 'definition', 'synonyms': ('Encephalon', 'Cerebro'), 'deprecated': None, 'predicates': {'comment': 'helpful misc'}, 'type': None, 'types': (), '_graph': None, 'source': <ontquery.plugins.services.InterLexRemote object at 0x7fb7d04abf28>})


# UPDATE

In [4]:
ixrt.update_entity?

Signature:
ixrt.update_entity(
    ilx_id: str = None,
    type: str = None,
    subThingOf: str = None,
    label: str = None,
    definition: str = None,
    synonyms=(),
    comment: str = None,
    predicates_to_add: dict = None,
    predicates_to_delete: dict = None,
)
Docstring: <no docstring>
File:      ~/Dropbox/git/ontquery/ontquery/plugins/services.py
Type:      method


In [8]:
from ilxutils.remotes import interlex_remote_test as ixrt
entity = dict(
    ilx_id = 'ILX:0738390',
    label = 'Offical label', # Can only one unique label per person
    type = 'term', # OPTIONS: term, annotation, relationship, cde, fde, pde
    definition = 'official definition',
    comment = 'helpful misc',
    # Optional
    subThingOf = '', # WARNING ::: must have at last '', can be blank but please fill this in if you can. 
    synonyms = ['Encephalon', 'Cerebro'],
    predicates_to_add = {}, # annotations and/or relationships to add
    predicates_to_delete = {}, # annotations and/or relationships to del
    # TODO: existing_ids will be an option later
)
result = ixrt.update_entity(**entity)
print(result)

[2019-10-11 13:47:28,619] -     INFO -       ontquery - interlex_client.py:796  - {'ilx_id': 'ILX:0738390', 'label': 'Offical label', 'type': 'term', 'definition': 'official definition', 'comment': 'helpful misc', 'superclass': '', 'synonyms': ['Encephalon', 'Cerebro']}


QueryResult({'iri': 'http://uri.interlex.org/base/ilx_0738390', 'curie': 'ILX:0738390', 'label': 'Offical label', 'labels': (), 'definition': 'official definition', 'synonyms': ('Encephalon', 'Cerebro'), 'deprecated': None, 'predicates': {'comment': 'helpful misc'}, 'type': None, 'types': (), '_graph': None, 'source': <ontquery.plugins.services.InterLexRemote object at 0x7fb7d04abf28>})


# PRODUCTION 
# BE CAREFUL PLEASE :)

In [3]:
# PRODUCTION 
from ilxutils.remotes import interlex_remote_production as ixr
# BE CAREFUL :)